# train_df_icdar

In [ ]:
OFFICIAL RELEASE: icdar_train_df_20250514_175905.csv

Code explanation: prendo i filenames ed estraggo same_text, writer e isEng. Creo un dataframe da questi che contiene anche il percorso del file.
Faccio il merge su writer con il dataframe da train_answers che ha writer e male come colonne.

In [ ]:
Setto un seed -> i risultati dovrebbero essere sempre gli stessi quando seleziono train e val ->
ho verificato che se lancio due volte in momenti diversi ottengo la stessa lista di writers in validation

X
sto prendendo writer, isEng etc dal nome del file e sto associando con le labels
prendo le labels da train_answers.csv

Problema: potrebbe essere che train_answers e train abbiano gli stessi scrittori  (handcrafted features da train è associato correttamente alle labels) 
ma che train_answers e i filenames delle immagini non seguano lo stesso ordinamento degli scrittori?
Come posso verificare che sto associando le immagini corrette?
Posso addestrare un modello che usi cnn. Se efficace non ci dovrebbero essere errori
posso fare un modello che matcha immagini e features
controllo da dove sono scaricati i dati
posso riestrarre le features e compararle con quelle già estratte

problema: potrebeb essere che train answers e train abbiano scrittori ordinati diversamente? Improbabile perchè sono tutti e due dataset di kaggle e FE funziona bene (80% val accuracy)



train_answers 

contiene 282 scrittori.
Number of times 'male' is 0: 143
Number of times 'male' is 1: 139

Dati estratti dai filenames

I load the images from the following folders: ['1_50', '51_100', '101_150', '151_200', '201_250', '251_300']

Costruzione del df da file:
Ho controllato che l'associazione con isEng sia corretta: pg 1 e 2 per uno scrittore sono in arabo, pg 3 e 4 in inglese

Come verificare se sto prendendo il valore giusto? Posso guardare degli esempi: 
-per writer=1 i primi due sono arabi isEng=0 e i secondi due sono inglesi isEng=1. Per writer 1 e writer 2 v=4 ha lo stesso testo mentre v=3 ha testo diverso. Per l'arabo v=2 hanno testo uguale (mi sembra) mentre v=1 ha testo diverso
-lo stesso vale per writer 253 e 254
-> sono ragionevolmente sicuro che sia tutto a posto

- Devo verificare che sto associando i percorsi giusti (che effettivamente nel df che creo writer e same_page siano costruiti nel modo giusto dal filename)
-> guardo il nome dei file e vedo se le colonne corrispondenti (writer, isEng etc ..) hanno il valore corretto. 
Ho controllato per 10 righe prese a caso e l'associazione è corretta

Ho aggiunto la colonna train
I need to check that 90% of writers is with train=1 -> the fractionof row with train=1 is exactly 90%
I need to check that a writer is only in one group -> i counted the unique values for each group of rows that share a writer. I have always one unique value 

Associating with the sex_df (adding the sex column)
- to confirm that the merging is working i randomly sample rows from sex_df and train_df and check that the writer and sex columns are shared -> confirmed

Test di verifica sul dataset finale ->
Number of times 'male' is 0: 572
Number of times 'male' is 1: 556
Number of times 'male' is something else: 0
All writers have both same_text=1 and same_text=0.
All writers have both isEng=1 and isEng=0.
Number of rows where train == 1: 1012
Fraction of rows where train == 1: 0.90
The train column is constant for all writers.
Each unique writer value occurs on exactly 4 rows.

I checked that there are no missing values:
All writers have both same_text=1 and same_text=0.
All writers have both isEng=1 and isEng=0.

ho verificato di avere 4 esempi per ogni scrittore

Prima di lanciare il codice ho fatto push su github
Salvo con seed e descrizione del dataset in un log file di tipo json 

# train_df_icdar_FE

OFFICIAL RELEASE: icdar_train_df_KAGGLE_20250514_181737.csv

Costruisce il df che posso usare per addestrare i modelli che usano le features di kaggle

Semplicemente fa il merging tra il train e il train_answer_df

# feature_extraction

RESULTS:
the validation accuracy I obtain by simply applying gbm to the kaggle features is 75% (100% on training). 
the validation on isEng is 98% -> task is much easier


In [ ]:
In this code I load the dataset generated by train_df_icdar_FE and apply without preprocessing some models like xgb etc ..

I simply need to check that the dataset has non nans, missing values etc .. -> checked and is allright

# patches_cc

In [ ]:
OFFICIAL RELEASE: icdar_train_df_patches_20250515_164130.csv

Carico il file generato da train_df_icdar. Per ogni documento singolo genero delle patches. Genero un nuovo dataframe identico a quello di partenza ma con 
le colonne x,y,x2,y2 per identificare le patches, assegno un valore univoco alle patches per further reference

NOT OPTIMAL
Devo verificare che le patches estratte abbiano senso -> tutte contengono testo ma alcune hanno il campo di vista mezzo vuoto

Devo verificare che ho 5 patch per ogni coppia isEng, same_page fissato lo scrittore
Dato che ogni coppia è associata a un indice unico controllo che per ogni indice ho 5 patches -> ok!

Devo verificare che le nuove patches create mantengano l'associazione con la label e gli altri parametri del df di input
-> estraggo delle righe a caso dal df esteso e vado a confrontare con le righe del writer corrispondente nel df originale

vedo che la riga i-esima che corrisponde ad una patch ha writer=n, male=m e train=t.
Le righe in train_df per writer=n hanno lo stesso m e t -> ragionevolmente sicuro che l'assoicazione è stata 
fatta correttamente

Devo verificare di avere 5 patches per ogni immagine di partenza -> verifico che ogni writer appaia 5*4 volte, che abbia sempre lo stesso valore di male ->ok

# deepfeatureextraction

OFFICIAL RELEASE:
trocr-small-stage1 (cls token only): icdar_EXTRACTED_train_df_trocr-small-stage1_20250516_122528.csv
trocr-small-stage1 (pooling): 
resnet50 (base transform): icdar_EXTRACTED_train_df_resnet50_20250516_161022.csv
resnet50 (custom transform):  icdar_EXTRACTED_train_df_resnet50_20250516_163737.csv
trocr-small-handwritten (cls token): icdar_EXTRACTED_train_df_trocr-small-handwritten_20250516_150814.csv
vit: icdar_EXTRACTED_train_df_vit-base-patch16-224-in21k_20250517_151642.csv
trocr-base-handwritten: icdar_EXTRACTED_train_df_trocr-base-handwritten_20250517_140220.csv
trocr-large-handwritten: icdar_EXTRACTED_train_df_trocr-large-handwritten_20250517_150651.csv
trocr-large-stage1:
clip-vit: icdar_EXTRACTED_train_df_clip-vit-large-patch14_20250517_144404.csv

This code takes a deep learning model and a dataframe with filenames (possibly with regions to crop)
For each row extracts the image from the filename, apply the model, returns the extracted features and appends them to the row

I'll need to check that the feature extractors work as expected

## Does the resnet work as expected?

## Does the trocr-small-stage1 work as expected?

When I load it it returns this: ''Some weights of VisionEncoderDecoderModel were not initialized from the model checkpoint at microsoft/trocr-small-stage1 and are newly initialized: ['encoder.pooler.dense.bias', 'encoder.pooler.dense.weight'] You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.''

what if i apply to data similar to its training data?

If i take model.encoder and use it for inference it returns me an output that has lasr_hidden_state and pooler_output as outputs
I cannot rely on the pooler output because the weights are not initialized 

If i subclass the model with: class FullClassifier(nn.Module):
    def __init__(self, encoder):
        super(FullClassifier, self).__init__()
        self.encoder = encoder.encoder
        self.embeddings = encoder.embeddings
        self.layernorm = encoder.layernorm
    def forward(self, x):
        hidden_state = self.embeddings(x)
        hidden_state= self.encoder(hidden_state).last_hidden_state
        hidden_state = self.layernorm(hidden_state)
        return hidden_state
I get the same output as the original model (if i take hidden_layers) -> in inference the last layer is the layernorm and subclassing is ok for inference

I have doubts that defining a new model in this way may cause problems in the gradient propagation -> since it is difficult to check I'll avoid subclassing as much as possible -> for sure i will use the base model for feature extraction

If i wrap the model with huggingface wrapper the output remains the same and it seems very unlikely to cause problems in the flow of gradients

## Does the trocr-small-handwriting work as expected?

I get the following warning: Some weights of VisionEncoderDecoderModel were not initialized from the model checkpoint at microsoft/trocr-small-handwritten and are newly initialized: ['encoder.pooler.dense.bias', 'encoder.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.

PROBLEM
I've done one test on the word-segmentation script. The word is correctly identified as "migration" (the problem was not the pooler since the same warning appeared)
If i feed an iam-lines example it works correctly. The processed image is VERY squeezed 
I am not sure if the most important factor for good recognition is the x,y ratio or the color distribution of the image -> I should select a bunch of crops with different ratios, transformations etc and compare the performance


PROBLEM
Chatgpt suggested the model outputs are different during inference and training: the model architecture i have when i print is the standard vision encoder decoder. trocr doesn't use the weights of the pooler in its inference even if they are in the architecture -> the architecture doesn't represent the forward method
I am checking this via printing hooks during inference. The hooks show that the pooler is not part of the inference (last layer is a layernorm)


## vit models


vit-base-patch16-224 : is pretrained on whole imagenet and fine tuned on old imagenet

vit-base-patch16-224-in21k: is pretrained but not finetuned
I can use the same checkpoint to load the model as it is or for image classification. If I load it for image classification the pooling layer is removed

## trocr-base and large

It takes almost 30min to upload the zipped files to colab

I have loaded the model using colab notebooks since my pc has too little ram to keep the model, even if compressed
The inference takes much less 9min instead of 30mins on my pc! (to build the dataset)

If i compare the number of patches that come out of the embedding layer and the number of patches in output i have 576 vs 577 -> it seems there is no distillation token (probably because it is the fine tuned version -> the distillation token is not used)

## clip-vit

# classifier on extracted features (mlp on feature extractor notebook)

Using gbm as classifier i get the following results (gender problem)
-transformers
-imagenet pretrained
vit-base-patch16-in21k: 67% val, 85% train 
clip-vit: 64%, 87%
-text pretrained
small-stage1: 60%, 80%
small-handwritten: 54%, 78%
base-handwritten: 61%, 82%
large-handwritten: 60%, 85%
vitstr_base: icdar_EXTRACTED_train_df_vitstr_base_20250524_001621.csv

cnns
-imagenet pretrained
resnet (standard encoding): 63%, 84%
resnet50 (custom): 61%, 85%
resnet50 (with pca to 384): 64%, 85%
-text pretrained
dresnet (resizing to 1024x1024): 63%, 84%
crnn (on resized images): 60%, 78%
crnn (cropping): 58%, 78%
crnn (padding): 58%, 80%




Using gbm classifier for language identification
small-handwritten: 77%, 88%

I would like to compare the performance on the gender detection and language detection tasks

I would like to compare the performance you get if you use cnn_features+kaggle_features+transformer_features vs only one model features

# preprocessing experiments

I should experiment with different input formats
1) crop wide rectangles from the images
2) crop line/sentence 
3) crop word level
4) crop character level
5) use full page -> i don't need any preprocessing. Simply apply deepfeature extraction to that dataset
6) random 32x32 crops and using them to build a single input
7) sub character crops (-> i use info about the shape of the line)
How much using full pages or smaller characters improves representations and so on ...
I'll test this on the small-handwriting, on the resnet model and on the vit.

how to get the crops?
I tried with projection profiles but i have to choose good parameters cause the lines are usually curved and superimpose
I didn't try with ccs but it may work well
I may try using keypoints extracted with sift or others.
I may try with ML models 

I'll do the following experiments:
1) crop smaller square sections (keeping the same number of patches per writer during training)
-> It will give useful information compared to the actual patches dataset
(i should test on different models)
2) crop words (or something similar to words)
-> i can understand if having meaningful text help learning
(I expect effects on the trocr models only)
3) crop random wide sections (comparable to average word width)
-> i can understand if the positive/negative effect comes from wordiness or proportion
(i expect effects on the trocr models and on the recognition models)
4) One dataset with more patches (eg same size of (1) but 3x more patches)
-> will see the effect of the number of crops per writer
5) One dataset with a big random crop from the center / paragrafi dai projection profiles / riquadro con tutto il testo dai projection profiles
-> will see the global vs local information importance
6) One dataset with half writers (decide the number of patches per writer from prev. experiments)
-> to get a feeling of the impact of the number of writers on learning

1) Extracting smaller patches: 1/100 of the area of the original image (icdar_train_df_patches_20250521_120324.csv)
vit: icdar_EXTRACTED_train_df_vit-base-patch16-224-in21k_20250523_170002.csv
trocr-base-handwritten: icdar_EXTRACTED_train_df_trocr-base-handwritten_20250523_173029.csv
resnet50 (standard): icdar_EXTRACTED_train_df_resnet50_20250523_175257.csv
2) Extracting words/sentences (icdar_train_df_words_sentences_20250522_230307.csv)
trocr-small-stage1: icdar_EXTRACTED_train_df_trocr-small-stage1_20250523_183350.csv
trocr-base-handwritten: icdar_EXTRACTED_train_df_trocr-base-handwritten_20250523_190523.csv
vit: icdar_EXTRACTED_train_df_vit-base-patch16-224-in21k_20250523_194225.csv
crnn: done on old
vitstr-base: icdar_EXTRACTED_train_df_vitstr_base_20250524_004825.csv
3) longer than higher (icdar_train_df_patches_20250522_234152.csv)
trocr-small-stage1 icdar_EXTRACTED_train_df_trocr-small-stage1_20250523_232141.cs
trocr-base-handwriting icdar_EXTRACTED_train_df_trocr-base-handwritten_20250523_233426.csv
crnn icdar_EXTRACTED_train_df_crnn_vgg16_bn_20250524_000208.csv
4) more patches (icdar_train_df_patches_20250522_235724.csv)
vit icdar_EXTRACTED_train_df_vit-base-patch16-224-in21k_20250523_203620.csv
trocr-base-handwritten: icdar_EXTRACTED_train_df_trocr-base-handwritten_20250523_213838.csv
resnet50: icdar_EXTRACTED_train_df_resnet50_20250523_221556.csv
5a) full pages
5b) crop the text region (icdar_train_df_body_20250523_181312.csv)
vit icdar_EXTRACTED_train_df_vit-base-patch16-224-in21k_20250523_224422.csv
trocr-base-handwritten icdar_EXTRACTED_train_df_trocr-base-handwritten_20250523_225140.csv
resnet50 (standard, resizing) icdar_EXTRACTED_train_df_resnet50_20250523_230400.csv

# research questions

## Do transformer architecture works? why? why not? how?

In the first experiments I cannot go above 60% accuracy with both vit and trocr
I didn't fine tune the whole model because it is time consuming -> part of the reason could be this
I did fine tune last transformer layers and it didn't work -> I am not sure if I've used the correct fine tuning approach. I am not sure I am using the pretrained model correctly for inference.
I did use features extracted from the last layer -> I am not sure if i am extracting the right features (considering the pooling layer or not? considering only cls ?)

For me is surprising that vit (general purpose) work as well as trocr -> should inquire

I should try using more information wrt the cls token only

I should try do a full fine tuning on a model

## what is the best feature extractor by default?

The different architectures also have a different transform/preprocessor. I saw that performance improves if I use a custom resnet transform instead of the 
standard one -> should I compare all models exploiting their own transform or the best transform for each?

Performance I reach wit handwriting pretrained cnn

Performance I reach with general pretrained cnns

Performance I reach with handwriting pretrained transformer models

Performance i reach with general purpose pretrained transformer models

I should chek wether architectures pretrained on text are better than the others if i change task (language detection)

I should split by task and compare: training on arab and classifying on arab, training on arab and classifying on english, ..

I should compare the accuracy on the arab and english samples and on the same and non-same samples -> to know what bring down accuracy

## how much does contrastive pretraining help fine tuning cnns?

# notes

My research problem is: experiment with alternative ways to solve the gender detection problem
A subproblem is: do transformer architectures work? why? why not?
Another subproblem is: pretrained cnns work. Can I find a technique to enhance their accuracy further?
Another subproblem is: do GNNs work?

I should test seriously the contrastive approach

I should try determining if pooling is better than cls or if there is a superior strategy that enables to retain as much of the transformer representation as possible (eg analyzing with cnns ..) -> save a full output from the models (eg for the handwriting model) and experiment

In [ ]:
I should experiment with other models pretrained on handwriting (eg cnn pretrained on handwriting)
layout analysis models: https://huggingface.co/microsoft/layoutlmv3-base 
text detection models: dbnet_resnet50
scene text recognition models( cnn based): crnn, sar, vitstr
formula recognition
---
to use text detection models I have to pool (since the outputs are from a fpn)-> https://chatgpt.com/share/682b4fdb-3370-8010-a3e9-d93d65bc93f5

sto testando i modelli di text recognition con 3 modalità di preprocessing delel immagini -> scelgo la migliore su crnn e poi applico agli altri due la stessa

I can test the models by using paddlepaddle models: https://chatgpt.com/share/682b4674-498c-8010-a9e1-c6eebf8f28c4 but it requires converting from their format
I can test the models by using doctr -> they are already in pytorch -> should be very easy to use

In [ ]:
I need to change the code for evaluating the results of classifiers on top of representations because I cannot make a discussion 
based on overfitted models. 

I should provide both the per patch and ensemble accuracy

I should evaluate both the accuracy and the roc/auc as in the icdar paper

In [ ]:
I have to rewrite the feature extraction code so that:
2) I can choose the task among: writer identification, gender detection, language detection -> do writer identification 
in another file. 
I save the characteristics of the training and the resulting metrics (not the model weights) in a table format i can anlyze with csv
I parallelize so that i can train using all my available cores

obiettivo: 
1) addestrare modelli che non overfittano
provare random forests
provare MLP con dropout
2) accelerare l'addestramento
parallelizzare il codice (applicando lo stesso codice a piu' files in contemporanea)
parallelizzare il training (eg con joblib)
utilizzare gpu per velocizzare training di MLP